In [1]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('timeseries-WGAN')+15]
import numpy as np
from scipy import stats
import math
import sys
sys.path.append(path+"/")
import random
import statsmodels.api as sm
from scipy.stats import norm

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import ArrowStyle
import japanize_matplotlib
import plotly
from scipy.stats import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

# dataset作成

In [72]:
data = np.array([0,1,2,3,4,5,6,7,8,9])

# dataloaderのクラスを作成

(X,Y)の組みとして、過去p時刻分の時系列をX, 今の時刻をYとするものを出力する。

In [73]:
class DataSet:
    def __init__(self, tsData, hat_p):
        """
        tsData：時系列データ（今回は一次元の時系列を想定）
        hat_p　：想定する次数
        initで入力データと出力データを作成する。
        """
        self.tsData =  torch.tensor(tsData, dtype=torch.float) # torch.tensorに変換
        self.hat_p = hat_p
        self.X, self.Y = self.make_XY(x=self.tsData, p=self.hat_p)
    
    def  __len__(self):
        return len(self.X) # データ数を返す
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    
    def make_XY(self, x:torch.tensor, p:int):
#         x = torch.tensor(x, dtype=torch.float)
        X = [x[i:i+p] for i in range(len(x)-p)]
        Y = [x[i+p] for i in range(len(x)-p)]
        return X, Y

In [85]:
hat_p=3
batch_size = 2

dataset = DataSet(data, hat_p=hat_p)# インスタンスの作成
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
# dataloaderをiterで回すと、毎回入力と出力のペアがlistでくる

In [89]:
for i in dataloader:
    print(i[0], i[1])

tensor([[1., 2., 3.],
        [6., 7., 8.]]) tensor([4., 9.])
tensor([[3., 4., 5.],
        [4., 5., 6.]]) tensor([6., 7.])
tensor([[2., 3., 4.],
        [0., 1., 2.]]) tensor([5., 3.])


batchでランダムに生成されたデータセットとして出力される。

### 実際に使う時はこのdataを使うね

In [2]:
phi_a = [1.0, -0.5, 0.7, -0.4]
phi_b = [1.0, -0.7]
p = len(phi_a)
q = len(phi_b)
dataSeed=5
N = 1000
data = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal')
inno = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal', return_innovation=True)